In [1]:
%load_ext autoreload
%reload_ext autoreload

### How to reroute neuron population into 3D map nerves

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from connectivity_graph import ConnectivityKnowledge
SCKAN_VERSION = 'sckan-2024-09-21'

npo = ConnectivityKnowledge(store_directory='production', sckan_version=SCKAN_VERSION, clean_connectivity=True)
logger.setLevel(logging.WARNING)

2024-11-26 17:29:02 [info     ] Map Knowledge version 0.20.4 with cache /Users/ymun794/Documents/MapCore/human-nerves/production/knowledgebase.db type=knowledge
2024-11-26 17:29:05 [info     ] With production SCKAN built at 2023-05-05 from https://scicrunch.org/api/1/sckan-scigraph type=knowledge


[2024-11-26 17:29:06,234] -  WARNING -     pyontutils -          core.py:818  - missed sentinel


2024-11-26 17:29:14 [info     ] With NPO built at 2024-09-21 from https://github.com/SciCrunch/NIF-Ontology/tree/sckan-2024-09-21, SHA: 289979cdad63ebc0d5843a07355f8b13ff3ae788 type=knowledge
2024-11-26 17:29:14 [info     ] Using knowledge source: sckan-2024-09-21-npo type=knowledge
2024-11-26 17:29:14 [info     ] Clearing connectivity knowledge for `sckan-2024-09-21-npo`... type=knowledge


Need to load 3D map nerve knowledge sources. For now, there are:
- nerve_point_annotations.json
    - this can be load as a json file
- M2.6_3D_whole-body.csv --> contains:
    - nerve ID
    - origins, pathways, landmarks, and destinations
    - laterality
    - all of these should be included in nerveAnnotations_manInBox.txt at each origin, waypoint, and destination
    - it is slightly modified from the original one by adding Superclass column

In [3]:
from routing import Rerouting, draw_entity, draw_waypoints
from connectivity_graph import display_connectivity_graph

rerouting = Rerouting('M2.6_3D_whole-body.csv', 'nerve_point_annotations.json', npo)

#### Reroute and draw ilxtr:neuron-type-bolew-unbranched-
This is an example of completely routed neuron population to 3d whole-body map

In [4]:
g = draw_entity(npo, rerouting, 'ilxtr:neuron-type-bolew-unbranched-12')
draw_waypoints(rerouting, g)

ilxtr:neuron-type-bolew-unbranched-12
SCKAN:
2024-11-26 13:04:35 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-bolew-unbranched-12 type=knowledge


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

Reroute for 3D Map:
2024-11-26 13:04:45 [info     ] Consulting SciCrunch for knowledge about FMA:53509 type=knowledge
2024-11-26 13:04:46 [info     ] Consulting SciCrunch for knowledge about FMA:50894 type=knowledge
2024-11-26 13:04:47 [info     ] Consulting SciCrunch for knowledge about FMA:53508 type=knowledge
2024-11-26 13:04:48 [info     ] Consulting SciCrunch for knowledge about FMA:50892 type=knowledge


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

#### Now checking all neuron population
The following script identifies all neuron population with the following criteria:
- at least one edge in the neuron population is detected in the 3d whole body map

However, this one edge can be just one point in the 3d whole body map.

**Note:** Running the following cell takes about 30 minutes

If you are only interested in the rendering results:
- Go to `Other complete examples` to see the other two complete mapping examples
- Go to `Example of partially rendered neuron population` to see the example of `good` partially complete example


In [5]:
from tqdm import tqdm

reroute_results = {}

for path in tqdm(npo.connectivity_paths()):
    if len((knowledge := rerouting.reroute_for_3d_map(path))['connectivity']) > 0:
        reroute_results[path] = {
            'original': npo.entity_knowledge(path),
            '3d_reroute': knowledge
        }


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [45:56<00:00,  5.90s/it]


In [6]:
### need to filter human neuron population only
human_reroute_results = {rr:kn for rr, kn in reroute_results.items() 
                         if len(taxons:=kn['original'].get('taxons', [])) == 0
                         or 'NCBITaxon:9606' in taxons                 # human
                         or 'NCBITaxon:40674' in taxons                # mammal
                        }

In [11]:
print("# of neuron populating where at least it's edge identified at 3d whole body map:", len(human_reroute_results))

# of neuron populating where at least it's edge identified at 3d whole body map: 34


#### Identifying neuron populations than can be rendered fully and partially 
Given that most of the neurons in the 3D whole body map have laterality, the minimum number of edges in the detected neuron population should be 4. Thus, there is at least one edge rendered in the left and right laterals of the 3D whole body map.

In [7]:
partially_mapped = [rr for rr, v in human_reroute_results.items() if len(v['3d_reroute']['connectivity'])>= 4]

print('# of partially mapped neuron population:', len(partially_mapped))
partially_mapped

# of partially mapped neuron population: 13


['ilxtr:neuron-type-aacar-4',
 'ilxtr:neuron-type-bolew-unbranched-18',
 'ilxtr:neuron-type-bolew-unbranched-16',
 'ilxtr:neuron-type-bolew-unbranched-19',
 'ilxtr:neuron-type-bolew-unbranched-17',
 'ilxtr:sparc-nlp/mmset2cn/1',
 'ilxtr:sparc-nlp/mmset2cn/3',
 'ilxtr:sparc-nlp/mmset1/10',
 'ilxtr:sparc-nlp/mmset1/5',
 'ilxtr:neuron-type-bolew-unbranched-11',
 'ilxtr:neuron-type-bolew-unbranched-12',
 'ilxtr:neuron-type-bolew-unbranched-9',
 'ilxtr:neuron-type-bolew-unbranched-10']

#### Other complete examples

In [8]:
g = draw_entity(npo, rerouting, 'ilxtr:sparc-nlp/mmset2cn/1')
draw_waypoints(rerouting, g)

ilxtr:sparc-nlp/mmset2cn/1
SCKAN:


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

Reroute for 3D Map:


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [9]:
g = draw_entity(npo, rerouting, 'ilxtr:sparc-nlp/mmset2cn/3')
draw_waypoints(rerouting, g)

ilxtr:sparc-nlp/mmset2cn/3
SCKAN:


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

Reroute for 3D Map:


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

#### Example of partially rendered neuron population

In [5]:
g = draw_entity(npo, rerouting, 'ilxtr:neuron-type-bolew-unbranched-9')
draw_waypoints(rerouting, g)

ilxtr:neuron-type-bolew-unbranched-9
SCKAN:
2024-11-26 13:05:00 [info     ] Consulting NPO for knowledge about ilxtr:neuron-type-bolew-unbranched-9 type=knowledge


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

Reroute for 3D Map:
2024-11-26 13:05:06 [info     ] Consulting SciCrunch for knowledge about UBERON:0035020 type=knowledge
2024-11-26 13:05:07 [info     ] Consulting SciCrunch for knowledge about FMA:53536 type=knowledge
2024-11-26 13:05:08 [info     ] Consulting SciCrunch for knowledge about UBERON:0035021 type=knowledge
2024-11-26 13:05:09 [info     ] Consulting SciCrunch for knowledge about FMA:53530 type=knowledge


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [12]:
npo.close()